In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import re

In [2]:
us_census_pop_wiki_link = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_historical_population'
request = requests.get(us_census_pop_wiki_link)
page = BeautifulSoup(request.text)

In [3]:
def select_pop_tables(tag):
    header = tag.find_previous_sibling(lambda tag: 'h' in tag.name)
    
    if tag.name != 'table' or not header:
        return False
    else:
        headline = header.select('.mw-headline')[0].text.lower()
        return ('population' in headline or 'census' in headline) and 'enslaved' not in headline

In [4]:
pop_tables = page.find_all(select_pop_tables)

In [5]:
year_col = []
state_col = []
pop_col = []

for table in pop_tables:
    headers = table.find_all('th')
    pop_rows = table.find_all('tr')[1:]
    
    for row in pop_rows:
        columns = row.find_all('td')
        
        state = ''
        for index, column in enumerate(columns):
            if 'name' in headers[index].text.lower():
                state = column.find('a').attrs['title']
            elif headers[index].text[:4].isdigit():
                state_col.append(state)
                year_col.append(headers[index].text[:4])
                pop_col.append(int(''.join(re.findall('\d+', column.text)) or '0'))

In [6]:
df = pd.DataFrame({
    'Year': year_col,
    'State': state_col,
    'Census population': pop_col
})

In [7]:
df.to_csv('../public_data/us_census_population.csv', index=False)

In [8]:
df.sample(20)

,Year,State,Census population
1104,1990,Maryland,4781468
938,1920,West Virginia,1463701
402,1810,Vermont,217713
556,1880,Georgia (U.S. state),1542180
1102,1970,Maryland,3922399
1093,2000,Louisiana,4468976
533,1920,Delaware,223003
677,1920,Michigan,3668412
1021,2000,Delaware,783600
220,1830,Missouri,140455
